In [1]:
import re
import xlrd
import pyvi
import numpy as np
import pandas as pd
from pyvi import ViTokenizer

loc = ("C:/Users/DELL/(K) Khanh_graduates_excellently/2. data_cleaning/raw_dataset.xlsx")
raw_data = xlrd.open_workbook(loc)

train_sheet = raw_data.sheet_by_index(0)
test_sheet = raw_data.sheet_by_index(1)

train_list1 = []
for i in range(train_sheet.nrows):
    train_list1.append(train_sheet.cell_value(i, 0))
test_list1 = []
for i in range(test_sheet.nrows):
    test_list1.append(test_sheet.cell_value(i, 0))

train_list2 = []
for x in train_list1:
    for y in x.split("< > *"):
        train_list2.append(y)
train_list3 = []
train_part_list = []
for i in range(len(train_list2)):
    if i%2 == 0:
        train_list3.append(train_list2[i])
    if i%2 != 0:
        train_part_list.append(train_list2[i])
train_list4 = []
for x in train_list3:
    for y in x.split(" < > "):
        train_list4.append(y)
train_part_list_scores = []
for i in range(len(train_list4)):
    if i%2 != 0:
        train_part_list_scores.append(train_list4[i])
    if i%2 == 0:
        pass

test_list2 = []
for x in test_list1:
    for y in x.split("< > *"):
        test_list2.append(y)
test_list3 = []
test_part_list = []
for i in range(len(test_list2)):
    if i%2 == 0:
        test_list3.append(test_list2[i])
    if i%2 != 0:
        test_part_list.append(test_list2[i])
test_list4 = []
for x in test_list3:
    for y in x.split(" < > "):
        test_list4.append(y)
test_part_list_scores = []
for i in range(len(test_list4)):
    if i%2 != 0:
        test_part_list_scores.append(test_list4[i])
    if i%2 == 0:
        pass    

space_replacing1 = re.compile("[^a-zA-Z_0-9ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹế\\s]")
space_replacing2 = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|(\n)")

def space_cleaning (reviews):
    cleaned_reviews = []
    for review in reviews:
        review = space_replacing1.sub(" ", str(review).lower())
        review = space_replacing2.sub(" ", str(review))
        review = " ".join(review.split())
        cleaned_reviews.append(review)
    return cleaned_reviews

train_clean = space_cleaning(train_part_list)
test_clean = space_cleaning(test_part_list)

def review_tokenizing (reviews):
    tokenized_reviews = []
    for review in reviews:
        tokenized_reviews.append(ViTokenizer.tokenize(review))
    return tokenized_reviews

tokenized_train = review_tokenizing(train_clean)
tokenized_test = review_tokenizing(test_clean)

stop_words = []
stop_words = [line.strip('\n') for line in open('vietnamese-stopwords-dash.txt', 'r', encoding='utf-8')]

def stop_words_removing (reviews):
    final = []
    for review in reviews: 
        removed = []
        for word in review.split(" "):
            if (word not in stop_words):
                removed.append(word)
        final.append(" ".join(removed))
    return final

final_train = stop_words_removing(tokenized_train)
final_test = stop_words_removing(tokenized_test)

print(final_train)
print(len(final_train))
print(train_part_list_scores)
print(len(train_part_list_scores))
print(" ")
print(" ")
print(" ")
print(final_test)
print(len(final_test))
print(test_part_list_scores)
print(len(test_part_list_scores))

['khách_sạn đẹp phòng sạch_sẽ nhân_viên nhiệt_tình lắm ổn', 'sai mường khen _giá _các nhân_viên cực_kì thân_thiện _mình đêm sn tặng bánh kem _sảnh rộng đẹp ngon sát biển _nội thất kk sạch_sẽ _đi nhân_viên cười giúp ko _khu vui_chơi trẻ sữa chữa hơi tiếc', 'ok', 'địa_điểm thuân tiện trang thiết_bị hiện_đại', 'ks sạch_sẽ nhân_viên nhiệt_tình tiền', 'cải_thiện thái_độ phục_vụ đội_ngũ nhân_viên bãi biển', 'phòng_ốc sạch_sẽ nhân_viên lịch_sự', 'đẹp giá_cả ok', 'ks phuc vụ dv ok ko dv bãi biển nhan viên ko kh ks ko hướng_dẫn m m mường miễn_cưỡng khăn hải_sản ko tùm_lum ko hài_lòng dv ng bãi biển thoi ok tàu xí xô xí xào suốt thiệt nhức sảnh', 'trung_tâm khách_sạn phòng hướng trực_diện biển tầm thoáng đẹp khách_sạn sang_trọng yên_tĩnh đông đoàn du_lịch tour nội_thất phòng bình_thường màu thảm trải sàn cảm_giác sạch_sẽ nội_thất xấu khách_sạn 5 có_nhân_viên đi đường tắm biển novotel ô_tô xe_máy chạy cửa khách_sạn đông lao rầm_rập đường biển sợ rớt tim nhân_viên thân_thiện trừ bộ_phận lễ_tân nhâ

In [3]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from bisect import bisect_left 

#1, Xây dựng bộ từ vựng

word_dict = []
for review in final_train:
    for word in review.split(" "):
        if word not in word_dict:
            word_dict.append(word)
for review in final_test:
    for word in review.split(" "):
        if word not in word_dict:
            word_dict.append(word)
print(word_dict)
print(len(word_dict))

['khách_sạn', 'đẹp', 'phòng', 'sạch_sẽ', 'nhân_viên', 'nhiệt_tình', 'lắm', 'ổn', 'sai', 'mường', 'khen', '_giá', '_các', 'cực_kì', 'thân_thiện', '_mình', 'đêm', 'sn', 'tặng', 'bánh', 'kem', '_sảnh', 'rộng', 'ngon', 'sát', 'biển', '_nội', 'thất', 'kk', '_đi', 'cười', 'giúp', 'ko', '_khu', 'vui_chơi', 'trẻ', 'sữa', 'chữa', 'hơi', 'tiếc', 'ok', 'địa_điểm', 'thuân', 'tiện', 'trang', 'thiết_bị', 'hiện_đại', 'ks', 'tiền', 'cải_thiện', 'thái_độ', 'phục_vụ', 'đội_ngũ', 'bãi', 'phòng_ốc', 'lịch_sự', 'giá_cả', 'phuc', 'vụ', 'dv', 'nhan', 'viên', 'kh', 'hướng_dẫn', 'm', 'miễn_cưỡng', 'khăn', 'hải_sản', 'tùm_lum', 'hài_lòng', 'ng', 'thoi', 'tàu', 'xí', 'xô', 'xào', 'suốt', 'thiệt', 'nhức', 'sảnh', 'trung_tâm', 'hướng', 'trực_diện', 'tầm', 'thoáng', 'sang_trọng', 'yên_tĩnh', 'đông', 'đoàn', 'du_lịch', 'tour', 'nội_thất', 'bình_thường', 'màu', 'thảm', 'trải', 'sàn', 'cảm_giác', 'xấu', '5', 'có_nhân_viên', 'đi', 'đường', 'tắm', 'novotel', 'ô_tô', 'xe_máy', 'chạy', 'cửa', 'lao', 'rầm_rập', 'sợ', 'rớt'

In [4]:
#2, Biểu diễn mỗi từ thành các one-hot-vector

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#gán index cho các từ trong từ điển bằng LabelEncoder
le = LabelEncoder()
le.fit(word_dict)
print('Words dict: ', le.classes_)
print(len(le.classes_))
# Biến đổi sang dạng số, mỗi từ trong từ điển được gán 1 số
x = le.transform(word_dict)
print('Words dict inverted into numbers: ', x)
print(len(x))
# Biến đổi lại sang class
print('Words dict inverted into classes: ', le.inverse_transform(x))
print(len(le.inverse_transform(x)))

Words dict:  ['' '0' '00' ... 'ứ_đọng' 'ứng' 'ứng_xử']
6336
Words dict inverted into numbers:  [2438 6181 3862 ... 2725 3955 6062]
6336
Words dict inverted into classes:  ['khách_sạn' 'đẹp' 'phòng' ... 'lotte' 'pv' 'đáo']
6336


In [5]:
#Thực hiện OneHotEncoder cho từ điển
oh = OneHotEncoder()
classes_indices = list(zip(le.classes_, np.arange(len(le.classes_))))
print('Words dict with indices: ,\n', classes_indices)
print(len(classes_indices))
print(" ")
print(" ")
oh.fit(classes_indices)
print('Arrays of categories and indices: ,\n', oh.categories_)
print(" ")
print(" ")
# Biến đổi từ điển sang dạng one-hot
words_indices = list(zip(word_dict, x))
print('Words dict inverted into corresponding indices: ,\n', words_indices)
print(len(words_indices))
print(" ")
one_hot = oh.transform(words_indices).toarray()
print('Words dict into one-hot matrices: \n', one_hot)
print(len(one_hot))
print(" ")
print('Words dict with indices inverted from one-hot matrices: \n', oh.inverse_transform(one_hot))
print(len(oh.inverse_transform(one_hot)))

Words dict with indices: ,
 [('', 0), ('0', 1), ('00', 2), ('000', 3), ('000d', 4), ('000vnd', 5), ('000vnf', 6), ('000đ', 7), ('01', 8), ('02', 9), ('03', 10), ('04', 11), ('05', 12), ('06', 13), ('07', 14), ('08', 15), ('09', 16), ('093', 17), ('0m7', 18), ('1', 19), ('10', 20), ('100', 21), ('1000', 22), ('1002', 23), ('1004', 24), ('100k', 25), ('100m', 26), ('101', 27), ('105', 28), ('108', 29), ('10am', 30), ('10c', 31), ('10cm', 32), ('10g', 33), ('10h', 34), ('10h30', 35), ('10k', 36), ('10km', 37), ('10m2', 38), ('10p', 39), ('10ph', 40), ('10phút', 41), ('10đ', 42), ('11', 43), ('114', 44), ('11g', 45), ('11h', 46), ('11h30', 47), ('11km', 48), ('11t', 49), ('12', 50), ('120', 51), ('1205', 52), ('120k', 53), ('120m2', 54), ('120usd', 55), ('129', 56), ('12cm', 57), ('12g', 58), ('12h', 59), ('12h00', 60), ('12m2', 61), ('12ng', 62), ('13', 63), ('130', 64), ('130k', 65), ('132b', 66), ('13h', 67), ('13h30', 68), ('14', 69), ('144', 70), ('14g', 71), ('14h', 72), ('14h00', 73

Words dict with indices inverted from one-hot matrices: 
 [['khách_sạn' 2438]
 ['đẹp' 6181]
 ['phòng' 3862]
 ...
 ['lotte' 2725]
 ['pv' 3955]
 ['đáo' 6062]]
6336


In [6]:
#3, Xây dựng word2vec trên bộ dữ liệu train

from gensim.models import Word2Vec
#word2vec bằng gensim, có thời gian sẽ làm cách chi tiết ko dùng gensim
sentences = [[item.lower() for item in doc.split()] for doc in final_train]
w2v_model = Word2Vec(sentences, min_count = 1, vector_size = 150, window = 10, sg = 1, workers = 8)
#min_count = int - Ignores all words with total absolute frequency lower than this.
#Change to using a model that always returns something for any word, like FastText – which will try to synthesize a vector for unknown words, using subwords learned during training.
#vector_size = int - Dimensionality of the feature vectors.
#window: Kích thước cửa sổ được sử dụng để khởi tạo các n-gram.
#sg: Nhận 2 giá trị {0, 1}. Nếu là 0: phương pháp CBOW, nếu là 1: skip-grams.
#wokers: Số core CPU được huy động để huấn luyện. Càng nhiều core tốc độ huấn luyện càng nhanh.
w2v_model.train(sentences, total_examples = w2v_model.corpus_count, epochs = 10)
#Một Epoch được tính là khi chúng ta đưa tất cả dữ liệu vào mạng neural network 1 lần.
#Khi dữ liệu quá lớn, chúng ta không thể đưa hết mỗi lần tất cả tập dữ liệu vào để huấn luyện được. Buộc lòng chúng ta phải chia nhỏ tập dữ liệu ra thành các batch (size nhỏ hơn),
#số thành phần (number of batch), mỗi phần có kích thước là batch size.

# Ex: Lấy véc tơ biểu diễn của từ "hồ"
print('Embedding vector shape: ', w2v_model.wv['hồ'].shape)
print(w2v_model.wv['hồ'])
print(" ")
# Ex: Lấy các từ có mối liên hệ gần nhất với 1 từ dựa trên khoảng cách
print(w2v_model.wv.most_similar(positive=['hồ']))
print(" ")
print(w2v_model.wv.similarity('hồ', 'bơi'))
print(w2v_model.wv.similarity('hồ', 'bếp'))

Embedding vector shape:  (150,)
[ 0.30470675  0.22849868  0.00966102 -0.38946894 -0.5989998   0.26611957
  0.50734746  0.33984157  0.01301824  0.41670766  0.27231407 -0.18037175
 -0.0865908   0.2333283  -0.16247484 -0.07329968 -0.01945311  0.27352166
 -0.06469201  0.03693653 -0.26260203 -0.17057514 -0.17061786  0.2934447
 -0.13106343 -0.02601197 -0.02731397 -0.45755276  0.23904698 -0.33426076
 -0.44493562 -0.05109684  0.10060892 -0.41712964  0.1004748   0.31809574
  0.76509285  0.31148413  0.025461    0.22526412 -0.3140123   0.12018937
 -0.00956797 -0.01636572  0.47343412 -0.20360346 -0.28314134  0.19539203
 -0.06325727  0.37053242 -0.32239074  0.30124706 -0.32787392 -0.06942879
  0.17612925 -0.34899962 -0.28194296 -0.08645126  0.49335214 -0.1987471
 -0.02402513 -0.57320005 -0.12697601 -0.1780187   0.20792234 -0.13715398
  0.02151814 -0.06532848 -0.4926087  -0.49007204  0.11484165  0.15910712
 -0.27326238  0.3760073   0.25940374  0.06462879  0.2220056   0.04247383
  0.38742384  0.11610

In [7]:
#rv_indices[rv_words.tolist().index('khách_sạn')]
rv_words=oh.categories_[0]
rv_indices=oh.categories_[1]

#chuyển final_train thành dạng số
reviews_in_number_train = []
for sentence in final_train:
    review_in_number = []
    for word in sentence.split(" "):
        review_in_number.append(rv_indices[rv_words.tolist().index(word)])
    reviews_in_number_train.append(review_in_number)
#chuyển final_test thành dạng số
reviews_in_number_test = []
for sentence in final_test:
    review_in_number= []
    for word in sentence.split(" "):
        review_in_number.append(rv_indices[rv_words.tolist().index(word)])
    reviews_in_number_test.append(review_in_number)

print(reviews_in_number_train)
print(len(reviews_in_number_train))
print(" ")
print(" ")
print(" ")
print(reviews_in_number_test)
print(len(reviews_in_number_test))

[[2438, 6181, 3862, 4407, 3498, 3449, 2871, 6325], [4202, 3138, 2382, 380, 378, 3498, 1416, 4681, 384, 6073, 4294, 5313, 638, 2360, 388, 4182, 6181, 3308, 4369, 545, 387, 4760, 2548, 4407, 390, 3498, 1306, 1849, 2553, 382, 5562, 5086, 4466, 1126, 2215, 4855], [3744], [6204, 4644, 4885, 4949, 4585, 2043], [2556, 4407, 3498, 3449, 4873], [1312, 4670, 3918, 6244, 3498, 656, 545], [3870, 4407, 3498, 2903], [6181, 1832, 3744], [2556, 3837, 5682, 1580, 3744, 2553, 1580, 656, 545, 3421, 5524, 2553, 2366, 2556, 2553, 2224, 2945, 2945, 3138, 3013, 2477, 2240, 2553, 5220, 2553, 2142, 1580, 3254, 656, 545, 4607, 3744, 5168, 5826, 5832, 5826, 5807, 4354, 4600, 3574, 4417], [5001, 2438, 3862, 2223, 5120, 545, 5286, 4612, 6181, 2438, 4220, 5887, 6098, 6027, 1561, 4921, 3720, 3862, 667, 3074, 4754, 5075, 4360, 1316, 4407, 3720, 5851, 2438, 268, 1256, 5990, 6126, 5306, 545, 3604, 5941, 5746, 1046, 1401, 2438, 6098, 2655, 4165, 6126, 545, 4451, 4184, 4832, 3498, 4681, 5117, 806, 2892, 3498, 1401, 2438,

In [13]:
loc = ("C:/Users/DELL/(K) Khanh_graduates_excellently/2. data_cleaning/raw_scores.xlsx")
raw_data = xlrd.open_workbook(loc)

#lấy sheet 1 của database
train_scores = raw_data.sheet_by_index(0)
#lấy sheet 2 của database
test_scores = raw_data.sheet_by_index(1)

#đưa dữ liệu sheet 1 vào trong một list
train_scores_list = []
for i in range(train_scores.nrows):
    train_scores_list.append(train_scores.cell_value(i, 0))
#đưa dữ liệu sheet 2 vào trong một list
test_scores_list = []
for i in range(test_scores.nrows):
    test_scores_list.append(test_scores.cell_value(i, 0))
    
#1 star=[1,0,0,0,0]; [0,1,0,0,0]; [0,0,1,0,0]; [0,0,0,1,0], [0,0,0,0,1]=5 stars
pad_score_train = []
for sc in train_scores_list:
    if 0 <= sc <= 3:
        pad_score_train.append([1,0,0,0,0])
    if 3 < sc <= 5:
        pad_score_train.append([0,1,0,0,0])
    if 5 < sc <= 7:
        pad_score_train.append([0,0,1,0,0])
    if 7 < sc <= 9:
        pad_score_train.append([0,0,0,1,0])
    if 9 < sc <= 10:
        pad_score_train.append([0,0,0,0,1])      
print(pad_score_train)
print(len(pad_score_train))

pad_score_test = []
for sc in test_scores_list:
    if 0 <= sc <= 3:
        pad_score_test.append([1,0,0,0,0])
    if 3 < sc <= 5:
        pad_score_test.append([0,1,0,0,0])
    if 5 < sc <= 7:
        pad_score_test.append([0,0,1,0,0])
    if 7 < sc <= 9:
        pad_score_test.append([0,0,0,1,0])
    if 9 < sc <= 10:
        pad_score_test.append([0,0,0,0,1])      
print(pad_score_test)
print(len(pad_score_test))

[[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 

In [15]:
#LSTM stage

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
# Set the maximum number of words per sentence (for both training and testing)
max_words = 50
# Pad sequences in train data and test data
pad_reviews_in_number_train = sequence.pad_sequences(reviews_in_number_train, maxlen=max_words)
pad_reviews_in_number_test = sequence.pad_sequences(reviews_in_number_test, maxlen=max_words)

print(pad_reviews_in_number_train)
print(len(pad_reviews_in_number_train))
print(" ")
print(pad_reviews_in_number_test)
print(len(pad_reviews_in_number_test))

[[   0    0    0 ... 3449 2871 6325]
 [   0    0    0 ... 1126 2215 4855]
 [   0    0    0 ...    0    0 3744]
 ...
 [   0    0    0 ...  594 3308 5688]
 [   0    0    0 ... 2773  967 3090]
 [   0    0    0 ...    0    0 2438]]
2300
 
[[   0    0    0 ... 1288  736 4462]
 [   0    0    0 ...    0    0 5001]
 [   0    0    0 ... 2871 1831 4873]
 ...
 [   0    0    0 ... 1342 4585 3720]
 [   0    0    0 ... 4681 3955 6062]
 [   0    0    0 ...  705 3377 2556]]
1200
